In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


sklearn version:  1.2.0


C:\Users\LG\anaconda3\envs\gpu\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


TF version:  2.12.0
No GPU was detected. CNNs can be very slow without a GPU.
GPU installed:  False


# 다. Baseline 학습 및 결과분석(Lenet5)

### 나) 부분에 생성된 데이터가 있다면 전체 다 실행 할 필요 없으나
### 생성된 데이터가 없고 파일 다) 만 있다는 가정하에 초반부에 나)의 내용도 포함시켜음

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import BatchNormalization, GlobalAveragePooling2D, MaxPooling2D, Add,  Dense, Conv2D, Activation
from keras.models import Sequential

In [ ]:
X_train=pd.read_csv('emnist-byclass-train.csv',header=None)
X_test=pd.read_csv('emnist-byclass-test.csv',header=None)

In [ ]:
def data_split(data_type_train,split_ratio=0.1):
    split = StratifiedShuffleSplit(n_splits=1, test_size=split_ratio, random_state=42)
    for train_idx, test_idx in split.split(data_type_train, data_type_train[0]):
        x_train = data_type_train.loc[train_idx]
        x_valid = data_type_train.loc[test_idx]
    return x_train,x_valid

In [ ]:
def data_load(data_type_train,data_type_valid,data_type_test):
    x_train =data_type_train.iloc[:, 1:].values
    y_train = data_type_train.iloc[:, 0].values
    x_valid =data_type_valid.iloc[:, 1:].values
    y_valid = data_type_valid.iloc[:, 0].values
    x_test = data_type_test.iloc[:, 1:].values
    y_test = data_type_test.iloc[:, 0].values
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    return x_train,y_train,x_valid,y_valid,x_test,y_test

In [ ]:
X_train,X_valid=data_split(X_train,0.235)

In [ ]:
X_train,y_train,X_valid,y_valid,X_test,y_test=data_load(X_train,X_valid,X_test)

In [ ]:
np.save('X_train',np.array(X_train,dtype=np.uint8))
np.save('y_train',np.array(y_train,dtype=np.uint8))
np.save('X_valid',np.array(X_valid,dtype=np.uint8))
np.save('y_valid',np.array(y_valid,dtype=np.uint8))
np.save('X_test',np.array(X_test,dtype=np.uint8))
np.save('y_test',np.array(y_test,dtype=np.uint8))

In [ ]:
import csv
def load_Emist():
    X_train=np.load('X_train.npy')
    y_train=np.load('y_train.npy')
    X_valid=np.load('X_valid.npy')
    y_valid=np.load('y_valid.npy')
    X_test=np.load('X_test.npy')
    y_test=np.load('y_test.npy')
    return X_train, y_train, X_valid, y_valid, X_test,y_test


In [ ]:
X_train, y_train, X_valid, y_valid, X_test, y_test = load_Emist()
X_train.shape,X_train.dtype

((533917, 28, 28, 1), dtype('uint8'))

In [ ]:
#data, batch size 성정
train_size=len(X_train)
valid_size=len(X_valid)
batch_size=32

In [ ]:
X_train=np.reshape(X_train,[-1,784])
X_valid=np.reshape(X_valid,[-1,784])

In [ ]:
y_train=np.reshape(y_train,[-1,1])
y_valid=np.reshape(y_valid,[-1,1])

In [ ]:
train_full = np.append(X_train,y_train,axis=1)
valid_full = np.append(X_valid,y_valid,axis=1)

In [ ]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    
    Emnist_dir = os.path.join("datasets", "Emnist")
    os.makedirs(Emnist_dir, exist_ok=True)
    path_format = os.path.join(Emnist_dir, "my_{}_{:02d}.csv")
    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        try:
            with open(part_csv, "xt", encoding="utf-8") as f:
                if header is not None:
                    f.write(header)
                    f.write("\n")
                for row_idx in row_indices:
                    f.write(",".join([repr(col) for col in data[row_idx]]))
                    f.write("\n")
        except:
            continue
    return filepaths

In [1]:
train_filepaths = save_to_multiple_csv_files(train_full, "train")
valid_filepaths = save_to_multiple_csv_files(valid_full, "valid")

### 나)에서  언급한대로 Lenet5에 맞는 preprocess로 바꿔줌

In [ ]:
n_inputs = X_train.shape[-1]
def preprocess_Lenet(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    x=tf.reshape(x,[28,28,1])
    y = tf.stack(fields[-1:])
    resized_image = tf.image.resize(x, [32, 32])
    final_image = resized_image/255.
    
    return final_image, y

In [ ]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess_Lenet, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [ ]:
train_set = csv_reader_dataset(train_filepaths,batch_size=batch_size, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths,batch_size=batch_size, repeat=None)

# optimizer

In [ ]:
#####LENET-5 정의
from tensorflow.keras import layers, models
model3 = models.Sequential()
model3.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(32,32,1)))
model3.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model3.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
model3.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model3.add(layers.Flatten())
model3.add(layers.Dense(120, activation='relu'))
model3.add(layers.Dense(84, activation='relu'))
model3.add(layers.Dense(62, activation='softmax'))

### adamW

In [ ]:

early_stopping = EarlyStopping(patience = 10)
data_name="byclass"
type_name="LN5"
checkpoint_callback = ModelCheckpoint(data_name+"_"+type_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

optimizer = tfa.optimizers.AdamW(weight_decay=1e-5, learning_rate=0.001,
                                 beta_1=0.9, beta_2=0.999)

model3.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
history3=model3.fit(train_set, batch_size = batch_size,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(valid_size / batch_size), 
                    epochs=1000,
                    callbacks=[early_stopping, checkpoint_callback])

In [21]:
early_stopping = EarlyStopping(patience = 10)
data_name="byclass"
type_name="LN5"
checkpoint_callback = ModelCheckpoint(data_name+"_"+type_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

optimizer = tfa.optimizers.AdamW(weight_decay=1e-5, learning_rate=0.001,
                                 beta_1=0.9, beta_2=0.999)

model3.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
history3=model3.fit(train_set, batch_size = batch_size,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(valid_size / batch_size), 
                    epochs=1000,
                    callbacks=[early_stopping, checkpoint_callback])

Epoch 1/1000
16678/16684 [============================>.] - ETA: 0s - loss: 0.5777 - accuracy: 0.8077
Epoch 1: val_loss improved from inf to 0.45623, saving model to byclass_LN5.h5
16684/16684 [==============================] - 134s 8ms/step - loss: 0.5776 - accuracy: 0.8077 - val_loss: 0.4562 - val_accuracy: 0.8379
Epoch 2/1000
16680/16684 [============================>.] - ETA: 0s - loss: 0.4259 - accuracy: 0.8468
Epoch 2: val_loss improved from 0.45623 to 0.41630, saving model to byclass_LN5.h5
16684/16684 [==============================] - 123s 7ms/step - loss: 0.4259 - accuracy: 0.8468 - val_loss: 0.4163 - val_accuracy: 0.8510
Epoch 3/1000
16681/16684 [============================>.] - ETA: 0s - loss: 0.3995 - accuracy: 0.8544
Epoch 3: val_loss improved from 0.41630 to 0.41325, saving model to byclass_LN5.h5
16684/16684 [==============================] - 103s 6ms/step - loss: 0.3996 - accuracy: 0.8544 - val_loss: 0.4133 - val_accuracy: 0.8513
Epoch 4/1000
16677/16684 [============

16684/16684 [==============================] - 136s 8ms/step - loss: 0.3450 - accuracy: 0.8690 - val_loss: 0.3858 - val_accuracy: 0.8590


### nesteroV

In [24]:
from tensorflow.keras import layers, models
model4 = models.Sequential()
model4.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(32,32,1)))
model4.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model4.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
model4.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model4.add(layers.Flatten())
model4.add(layers.Dense(120, activation='relu'))
model4.add(layers.Dense(84, activation='relu'))
model4.add(layers.Dense(62, activation='softmax'))

early_stopping2 = EarlyStopping(patience = 10)
data_name="byclass"
type_name="LN5"
checkpoint_callback2 = ModelCheckpoint(data_name+"_"+type_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#nesteroV 사용해보기
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9,
                                    nesterov=True)

model4.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
history4=model4.fit(train_set, batch_size = batch_size,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(valid_size / batch_size), 
                    epochs=1000,
                    callbacks=[early_stopping2, checkpoint_callback2])

Epoch 1/1000
16681/16684 [============================>.] - ETA: 0s - loss: 0.8855 - accuracy: 0.7368
Epoch 1: val_loss improved from inf to 0.57336, saving model to byclass_LN5.h5
16684/16684 [==============================] - 118s 7ms/step - loss: 0.8854 - accuracy: 0.7368 - val_loss: 0.5734 - val_accuracy: 0.8080
Epoch 2/1000
16681/16684 [============================>.] - ETA: 0s - loss: 0.5216 - accuracy: 0.8220
Epoch 2: val_loss improved from 0.57336 to 0.50539, saving model to byclass_LN5.h5
16684/16684 [==============================] - 118s 7ms/step - loss: 0.5216 - accuracy: 0.8220 - val_loss: 0.5054 - val_accuracy: 0.8243
Epoch 3/1000
16683/16684 [============================>.] - ETA: 0s - loss: 0.4761 - accuracy: 0.8338
Epoch 3: val_loss improved from 0.50539 to 0.46884, saving model to byclass_LN5.h5
16684/16684 [==============================] - 118s 7ms/step - loss: 0.4762 - accuracy: 0.8338 - val_loss: 0.4688 - val_accuracy: 0.8362
Epoch 4/1000
16679/16684 [============

16684/16684 [==============================] - 90s 5ms/step - loss: 0.3494 - accuracy: 0.8691 - val_loss: 0.3851 - val_accuracy: 0.8594
Epoch 28/1000
16673/16684 [============================>.] - ETA: 0s - loss: 0.3408 - accuracy: 0.8713
Epoch 28: val_loss did not improve from 0.38323
16684/16684 [==============================] - 89s 5ms/step - loss: 0.3408 - accuracy: 0.8713 - val_loss: 0.3876 - val_accuracy: 0.8578
Epoch 29/1000
16677/16684 [============================>.] - ETA: 0s - loss: 0.3378 - accuracy: 0.8719
Epoch 29: val_loss did not improve from 0.38323
16684/16684 [==============================] - 89s 5ms/step - loss: 0.3378 - accuracy: 0.8719 - val_loss: 0.3878 - val_accuracy: 0.8594
Epoch 30/1000
16673/16684 [============================>.] - ETA: 0s - loss: 0.3421 - accuracy: 0.8708
Epoch 30: val_loss did not improve from 0.38323
16684/16684 [==============================] - 89s 5ms/step - loss: 0.3421 - accuracy: 0.8708 - val_loss: 0.3871 - val_accuracy: 0.8603
Epo

### adam

In [ ]:
#####LENET-5 정의
from tensorflow.keras import layers, models
model3 = models.Sequential()
model3.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(32,32,1)))
model3.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model3.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
model3.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model3.add(layers.Flatten())
model3.add(layers.Dense(120, activation='relu'))
model3.add(layers.Dense(84, activation='relu'))
model3.add(layers.Dense(62, activation='softmax'))

In [ ]:
early_stopping = EarlyStopping(patience = 10)
data_name="byclass"
type_name="LN5"
checkpoint_callback = ModelCheckpoint(data_name+"_"+type_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
history3=model3.fit(train_set, batch_size = batch_size,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(valid_size / batch_size), 
                    epochs=1000,
                    callbacks=[early_stopping, checkpoint_callback])

Epoch 1/1000
16681/16684 [============================>.] - ETA: 0s - loss: 0.5739 - accuracy: 0.8085
Epoch 1: val_loss improved from inf to 0.45191, saving model to byclass_LN5.h5
16684/16684 [==============================] - 96s 6ms/step - loss: 0.5738 - accuracy: 0.8085 - val_loss: 0.4519 - val_accuracy: 0.8407
Epoch 2/1000
16680/16684 [============================>.] - ETA: 0s - loss: 0.4172 - accuracy: 0.8492
Epoch 2: val_loss improved from 0.45191 to 0.41200, saving model to byclass_LN5.h5
16684/16684 [==============================] - 94s 6ms/step - loss: 0.4172 - accuracy: 0.8492 - val_loss: 0.4120 - val_accuracy: 0.8526
Epoch 3/1000
16683/16684 [============================>.] - ETA: 0s - loss: 0.3893 - accuracy: 0.8569
Epoch 3: val_loss improved from 0.41200 to 0.40406, saving model to byclass_LN5.h5
16684/16684 [==============================] - 94s 6ms/step - loss: 0.3893 - accuracy: 0.8569 - val_loss: 0.4041 - val_accuracy: 0.8553
Epoch 4/1000
16676/16684 [===============

# activation

### selu

In [3]:
from tensorflow.keras import layers, models
model2 = models.Sequential()
model2.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), padding='same',activation='selu', input_shape=(32,32,1)))
model2.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model2.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='selu'))
model2.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model2.add(layers.Flatten())
model2.add(layers.Dense(120, activation='selu'))
model2.add(layers.Dense(84, activation='selu'))
model2.add(layers.Dense(62, activation='softmax'))

In [ ]:
early_stopping_BRN50 = EarlyStopping(patience = 5)
data_name="byclass" 
type_name="lenet_selu"
checkpoint_callback_BRN50 = ModelCheckpoint(data_name+"_"+type_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
model2.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
history=model2.fit(train_set, batch_size = batch_size,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(valid_size / batch_size), 
                    epochs=1000,
                    callbacks=[early_stopping_BRN50, checkpoint_callback_BRN50])

Epoch 1/1000
16677/16684 [============================>.] - ETA: 0s - loss: 0.9342 - accuracy: 0.7138
Epoch 1: val_loss improved from inf to 0.55519, saving model to byclass_lenet_selu.h5
16684/16684 [==============================] - 70s 4ms/step - loss: 0.9342 - accuracy: 0.7138 - val_loss: 0.5552 - val_accuracy: 0.8099
Epoch 2/1000
16676/16684 [============================>.] - ETA: 0s - loss: 0.7674 - accuracy: 0.7551
Epoch 2: val_loss improved from 0.55519 to 0.53469, saving model to byclass_lenet_selu.h5
16684/16684 [==============================] - 68s 4ms/step - loss: 0.7674 - accuracy: 0.7551 - val_loss: 0.5347 - val_accuracy: 0.8164
Epoch 3/1000
16671/16684 [============================>.] - ETA: 0s - loss: 0.7373 - accuracy: 0.7640
Epoch 3: val_loss improved from 0.53469 to 0.51580, saving model to byclass_lenet_selu.h5
16684/16684 [==============================] - 66s 4ms/step - loss: 0.7372 - accuracy: 0.7640 - val_loss: 0.5158 - val_accuracy: 0.8217
Epoch 4/1000
16679/1

### elu

In [ ]:
model3 = models.Sequential()
model3.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='elu', input_shape=(32,32,1)))
model3.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model3.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='elu'))
model3.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model3.add(layers.Flatten())
model3.add(layers.Dense(120, activation='elu'))
model3.add(layers.Dense(84, activation='elu'))
model3.add(layers.Dense(62, activation='softmax'))

In [ ]:
early_stopping_BRN50 = EarlyStopping(patience = 5)
data_name="byclass" 
type_name="lenet_elu"
checkpoint_callback_BRN50 = ModelCheckpoint(data_name+"_"+type_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
model3.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
history3=model3.fit(train_set, batch_size = batch_size,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(valid_size / batch_size), 
                    epochs=1000,
                    callbacks=[early_stopping_BRN50, checkpoint_callback_BRN50])

Epoch 1/1000
16682/16684 [============================>.] - ETA: 0s - loss: 0.8126 - accuracy: 0.7477
Epoch 1: val_loss improved from inf to 0.47137, saving model to byclass_lenet_elu.h5
16684/16684 [==============================] - 215s 13ms/step - loss: 0.8125 - accuracy: 0.7477 - val_loss: 0.4714 - val_accuracy: 0.8372
Epoch 2/1000
16680/16684 [============================>.] - ETA: 0s - loss: 0.6068 - accuracy: 0.8011
Epoch 2: val_loss improved from 0.47137 to 0.45647, saving model to byclass_lenet_elu.h5
16684/16684 [==============================] - 220s 13ms/step - loss: 0.6068 - accuracy: 0.8011 - val_loss: 0.4565 - val_accuracy: 0.8418
Epoch 3/1000
16682/16684 [============================>.] - ETA: 0s - loss: 0.5723 - accuracy: 0.8109
Epoch 3: val_loss improved from 0.45647 to 0.45073, saving model to byclass_lenet_elu.h5
16684/16684 [==============================] - 219s 13ms/step - loss: 0.5723 - accuracy: 0.8109 - val_loss: 0.4507 - val_accuracy: 0.8447
Epoch 4/1000
1668